## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:

Please visit https://sender.net/

Create a free account.

Then generate an API access token:

Settings (left sidebar) >> API access tokens >> Create token (set validity as needed)

Copy the token to your .env file:

`SENDER_API_TOKEN=xxxx`

Also set sender and recipient emails (adjust to your verified addresses):

`SENDER_FROM_EMAIL=you@yourdomain.com`
`SENDER_TO_EMAIL=recipient@example.com`

Docs: Sender.net Access Tokens (help): https://help.sender.net/knowledgebase/access-tokens/
API overview: https://help.sender.net/knowledgebase/api-documentation/

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import requests
import os
import asyncio



In [2]:
load_dotenv(override=True)

True

In [3]:
# Let's test push notifications instead of email

def send_test_notification():
    pushover_user = os.environ.get('PUSHOVER_USER')
    pushover_token = os.environ.get('PUSHOVER_TOKEN')
    
    if not pushover_user or not pushover_token:
        raise RuntimeError("Missing PUSHOVER_USER or PUSHOVER_TOKEN in environment")
    
    def push(message):
        print(f"Push: {message}")
        payload = {"user": pushover_user, "token": pushover_token, "message": message}
        response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
        return response
    
    message = "Test notification: This is an important test from the sales system"
    response = push(message)
    
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Notification sent successfully!")
    else:
        print("❌ Notification failed to send")
        print(f"Response: {response.text}")

send_test_notification()

Push: Test notification: This is an important test from the sales system
Status: 200
✅ Notification sent successfully!


### Did you receive the test email

If you get a 2xx response (e.g., 200 or 202), then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

- Double-check your `SENDER_API_TOKEN` and that your `SENDER_FROM_EMAIL` is valid/verified in Sender.net.
- See Sender.net API docs: https://help.sender.net/knowledgebase/api-documentation/

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Enhance Your SOC 2 Compliance with Our AI-Powered Solution

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in streamlining the SOC 2 compliance process for businesses like yours.

In today’s fast-paced environment, ensuring compliance can be both time-consuming and complex. Our AI-powered software simplifies this process, helping you automate documentation, identify compliance gaps, and prepare for audits with ease.

With ComplAI, you can:

- **Reduce manual workload:** Automate repetitive tasks and focus on what matters most.
- **Stay updated:** Benefit from real-time compliance insights and automated reports.
- **Prepare effortlessly:** Ensure you’re always audit-ready with our comprehensive audit trail features.

I’d love to schedule a brief call to discuss how ComplAI can support your compliance efforts and help you save time and resources. Would next week work for you?

Thank you f

[non-fatal] Tracing: server error 500, retrying.


In [7]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a leading provider of SaaS solutions that streamline the SOC 2 compliance process.

In today’s competitive landscape, maintaining robust compliance and preparing for audits can be both time-consuming and complex. Our AI-powered platform is designed to simplify these tasks, allowing you to focus on what really matters—growing your business.

With ComplAI, you can:

- **Automate Documentation**: Efficiently manage policies and procedures, reducing manual effort.
- **Real-Time Monitoring**: Stay ahead of potential compliance issues with continuous monitoring.
- **Audit Readiness**: Ensure you're always prepared for an audit with tools that centralize evidence and reports.

I would love the opportunity to discuss how ComplAI can specifically benefit [Recipient’s Company Name] and help you enhance your compliance efforts. 

In [8]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [9]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Are You Ready for Your SOC2 Audit? Let’s Not Make It a Horror Story! 👻

Hey [Recipient's Name],

I hope this email finds you well—and not buried under a mountain of compliance documents! 

Imagine this: It’s audit season, and instead of panicking like a cat in a bathtub, you’re confidently sipping your coffee while our AI-powered tool takes care of your SOC2 compliance. Sounds dreamy, right? ☕✨

At ComplAI, we believe compliance shouldn’t be a dreaded rollercoaster ride but more like a leisurely stroll in the park. With our SaaS solution, you’ll be able to streamline your processes, keep track of documentation, and sail through audits like a seasoned pro—minus the stress sweat!

Here’s what makes us special:
- **Automated Document Tracking**: Because who wants to play hide-and-seek with files?
- **Real-time Compliance Monitoring**: No more waiting until the last minute to avoid those “surprise” moments.
- **AI-Powered Insights**: Think of it as having a compl

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [10]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [11]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [12]:
@function_tool
def send_notification(body: str):
    """ Send out a push notification with the given body to the sales team """
    pushover_user = os.environ.get('PUSHOVER_USER')
    pushover_token = os.environ.get('PUSHOVER_TOKEN')
    
    if not pushover_user or not pushover_token:
        raise RuntimeError("Missing PUSHOVER_USER or PUSHOVER_TOKEN in environment")

    def push(message):
        print(f"Push: {message}")
        payload = {"user": pushover_user, "token": pushover_token, "message": message}
        response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
        return response

    response = push(body)
    response.raise_for_status()
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [14]:
# Let's look at it
send_notification

FunctionTool(name='send_notification', description='Send out a push notification with the given body to the sales team', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_notification_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10e3ba8e0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [15]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed8da80>, strict_json_schema=True, is_enabled=True)

In [17]:
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write a cold sales email")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a cold sales email")

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [18]:
tools = [tool1, tool2, tool3, send_notification]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed8da80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed8f100>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

## And now it's time for our Sales Manager - our planning agent

In [19]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_notification tool to send the best email (and only the best email) as a push notification to the sales team.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE notification using the send_notification tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

Push: Subject: Let’s Get Your SOC2 Compliance on Point—Without Losing Your Hair!

Dear CEO,

I hope this email finds you in high spirits and with enough coffee to conquer the world! ☕️

Let’s be real—navigating the murky waters of SOC2 compliance can feel like trying to teach a cat to fetch. But fear not! At ComplAI, we’ve engineered an AI superpower to make compliance as easy as pie (the one you wish you could eat without any calories).

Imagine a world where audits feel as relaxing as a Sunday morning stroll, and compliance becomes your organization’s secret weapon. With our cutting-edge SaaS solution, you can put your worries on mute and keep your focus on that all-important growth strategy (and maybe even plan that beach vacation you’ve been dreaming of!).

Can we hop on a quick call to chat about how we can turn your compliance headaches into a success story? After all, who doesn’t want their compliance issues handled faster than a pizza delivery?

Looking forward to hearing from 

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the Sender.net website, to hunt for clues. Make sure your API token and sender are valid. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [20]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [21]:
@function_tool
def send_notification_with_title(subject: str, message: str) -> Dict[str, str]:
    """ Send out a push notification with the given subject and message to the sales team """
    pushover_user = os.environ.get('PUSHOVER_USER')
    pushover_token = os.environ.get('PUSHOVER_TOKEN')
    
    if not pushover_user or not pushover_token:
        raise RuntimeError("Missing PUSHOVER_USER or PUSHOVER_TOKEN in environment")

    def push(title, body):
        print(f"Push notification - Title: {title}, Message: {body}")
        payload = {
            "user": pushover_user, 
            "token": pushover_token, 
            "title": title,
            "message": body
        }
        response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
        return response

    response = push(subject, message)
    response.raise_for_status()
    return {"status": "success"}

In [22]:
tools = [subject_tool, html_tool, send_notification_with_title]

In [23]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10f221580>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10f220a40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_notification_with_title', description='Send out a push notification with the given subject and mes

In [24]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_notification_with_title tool to send a push notification with the subject and message body to the sales team."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it as a push notification")

### Now we have 3 tools and 1 handoff

In [25]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed8da80>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ed8f100>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [26]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending as a push notification to the sales team.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Push notification - Title: SOC2 Compliance: The *Fun* Side of Audits 🎉, Message: Hey [CEO’s Name],

I hope this email finds you thriving! I’m Alice from ComplAI, and I promise I’m not here to put you to sleep with compliance talk.

Let’s face it, audits can feel about as fun as watching paint dry. But what if I told you that our AI-powered tool can turn SOC2 compliance into an experience you’ll actually enjoy? (Okay, maybe not a roller coaster, but close!)

With ComplAI, you can automate those boring tasks and wave goodbye to audit stress. Imagine sipping your coffee while our intelligent system does the heavy lifting—because why not let AI take the wheel?

I’d love to chat about how we can sprinkle a little magic on your compliance journey. Up for a quick call? I promise there’ll be no audit slides involved!

Cheers,
Alice ✨
ComplAI Team


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have Sender.net call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.